<a href="https://colab.research.google.com/github/katiacardoso/Selector_Hat/blob/main/DecisionTree_again.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [449]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import random

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV

'''# Suponha que você tenha um DataFrame chamado df com as colunas 'habilidade', 'interesse', 'personalidade' e 'casa'
df = pd.DataFrame({
    'habilidade': ['alta', 'baixa', 'media', 'alta', 'baixa'],
    'interesse': ['magia', 'esportes', 'estudos', 'magia', 'estudos'],
    'personalidade': ['extrovertido', 'introvertido', 'extrovertido', 'introvertido', 'extrovertido'],
    'casa': ['Grifinória', 'Lufa-Lufa', 'Corvinal', 'Grifinória', 'Sonserina']
})'''

# Defina as possíveis opções para cada coluna
habilidades = ['alta', 'media', 'baixa']
interesses = ['magia', 'esportes', 'estudos']
personalidades = ['extrovertido', 'introvertido']
casas = ['Grifinória', 'Lufa-Lufa', 'Corvinal', 'Sonserina']

# Defina o número de exemplos que você deseja gerar
n_exemplos = 200000

# Crie listas vazias para cada coluna
habilidade_lista = []
interesse_lista = []
personalidade_lista = []
casa_lista = []

# Use um loop for para gerar os dados
for _ in range(n_exemplos):
    habilidade_lista.append(random.choice(habilidades))
    interesse_lista.append(random.choice(interesses))
    personalidade_lista.append(random.choice(personalidades))
    casa_lista.append(random.choice(casas))

# Crie o DataFrame
df = pd.DataFrame({
    'habilidade': habilidade_lista,
    'interesse': interesse_lista,
    'personalidade': personalidade_lista,
    'casa': casa_lista
})


df.to_csv('rf.csv', index=False)

#print(df, '\n')



#df.head()

# Crie um LabelEncoder para cada coluna
le_habilidade = LabelEncoder().fit(['alta', 'media', 'baixa'])
le_interesse = LabelEncoder().fit(['magia', 'esportes', 'estudos'])
le_personalidade = LabelEncoder().fit(['extrovertido', 'introvertido'])
le_casa = LabelEncoder().fit(['Grifinória', 'Lufa-Lufa', 'Corvinal', 'Sonserina'])

# Transforme os dados usando os LabelEncoders
df['habilidade'] = le_habilidade.transform(df['habilidade'])
df['interesse'] = le_interesse.transform(df['interesse'])
df['personalidade'] = le_personalidade.transform(df['personalidade'])
df['casa'] = le_casa.transform(df['casa'])

# Divida os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df[['habilidade', 'interesse', 'personalidade']], df['casa'], test_size=0.3)



from imblearn.over_sampling import RandomOverSampler

# Crie o oversampler
ros = RandomOverSampler(random_state=42)

# Ajuste e aplique o oversampler
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

# Agora, X_resampled e y_resampled são seus dados balanceados

# Conte o número de ocorrências de cada classe
contagem = np.bincount(y_resampled)

# Imprima a contagem para cada casa
#for casa, cont in zip(le_casa.classes_, contagem):
#    print(f"A casa {casa} tem {cont} alunos após o balanceamento.")



# Defina os hiperparâmetros a serem ajustados
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Crie o objeto GridSearchCV
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=5)

# Ajuste o GridSearchCV aos dados de treinamento
grid_search.fit(X_train, y_train)

# Imprima os melhores hiperparâmetros
#print(grid_search.best_params_)

# Use o melhor modelo para fazer previsões
model = grid_search.best_estimator_



In [450]:
# Obtenha as probabilidades para um novo aluno
novo_aluno = np.array([
    le_habilidade.transform(['alta'])[0],
    le_interesse.transform(['magia'])[0],
    le_personalidade.transform(['extrovertido'])[0]
]).reshape(1, -1)

casa_prevista = le_casa.inverse_transform(model.predict(novo_aluno))
print(casa_prevista)


probabilidades = model.predict_proba(novo_aluno)

# Imprima as probabilidades para cada casa
for casa, probabilidade in zip(le_casa.classes_, probabilidades[0]):
    print(f"A probabilidade do novo aluno pertencer à casa {casa} é {probabilidade}")

['Lufa-Lufa']
A probabilidade do novo aluno pertencer à casa Corvinal é 0.2473328129065834
A probabilidade do novo aluno pertencer à casa Grifinória é 0.250975800156128
A probabilidade do novo aluno pertencer à casa Lufa-Lufa é 0.2526671870934166
A probabilidade do novo aluno pertencer à casa Sonserina é 0.24902419984387197


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [432]:
# Obtenha a importância das características
importances = model.feature_importances_

# Crie um DataFrame para exibir as importâncias
df = pd.DataFrame({
    'Característica': ['habilidade', 'interesse', 'personalidade'],
    'Importância': importances
})

# Ordene o DataFrame por importância
df = df.sort_values(by='Importância', ascending=False)

# Imprima o DataFrame
print(df)

  Característica  Importância
1      interesse     0.401613
0     habilidade     0.330645
2  personalidade     0.267742


In [451]:

# Faça previsões no conjunto de teste
y_pred = model.predict(X_test)

# Calcule a acurácia
acuracia = accuracy_score(y_test, y_pred)
print(acuracia)
#ou
acuracia = model.score(X_test, y_test)
print(acuracia)


unique_classes = np.unique(np.concatenate((y_test, y_pred)))
relatorio = classification_report(y_test, y_pred, target_names=le_casa.inverse_transform(unique_classes))
print(relatorio)

# Agora você pode usar inverse_transform para obter os rótulos originais
X_train['habilidade'] = le_habilidade.inverse_transform(X_train['habilidade'])
X_train['interesse'] = le_interesse.inverse_transform(X_train['interesse'])
X_train['personalidade'] = le_personalidade.inverse_transform(X_train['personalidade'])

X_test['habilidade'] = le_habilidade.inverse_transform(X_test['habilidade'])
X_test['interesse'] = le_interesse.inverse_transform(X_test['interesse'])
X_test['personalidade'] = le_personalidade.inverse_transform(X_test['personalidade'])

y_train = le_casa.inverse_transform(y_train)
y_test = le_casa.inverse_transform(y_test)

# Agora você pode visualizar os dados de treinamento e teste com os rótulos originais
print("Dados de treinamento:")
print(X_train)
print(y_train)

print("\nDados de teste:")
print(X_test)
print(y_test)



0.2486
0.2486
              precision    recall  f1-score   support

    Corvinal       0.25      0.17      0.20     15106
  Grifinória       0.24      0.05      0.09     14992
   Lufa-Lufa       0.25      0.56      0.34     14890
   Sonserina       0.25      0.22      0.23     15012

    accuracy                           0.25     60000
   macro avg       0.25      0.25      0.22     60000
weighted avg       0.25      0.25      0.22     60000

Dados de treinamento:
       habilidade interesse personalidade
139958      baixa     magia  introvertido
139135      baixa  esportes  introvertido
7576        baixa  esportes  extrovertido
8663        baixa  esportes  introvertido
140134      baixa  esportes  extrovertido
...           ...       ...           ...
80558        alta     magia  extrovertido
195444      media   estudos  introvertido
12277       baixa   estudos  extrovertido
67946       baixa   estudos  extrovertido
2556        media  esportes  introvertido

[140000 rows x 3 columns